In [1]:
import gym
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib as plt


from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#wrapper for cont env, with DDPG, plot result for several steps
def ddpg_eval_interval(p, L, t_t, n_iter, learning_rate=0.0003):
    
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    cont_env = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    print("Running DDPG w/: p=", p, ", L=",L)
    n_actions = cont_env.action_space.shape[-1]
    action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
    
    cont_model = DDPG("MlpPolicy", cont_env, action_noise=action_noise, verbose=1, gamma = 1)
    
    env_eval = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=100)#
        timesteps = timesteps + 100

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        print(res_mean_arr)
    
#     plt.pyplot.plot(res_mean_arr)

    return res_mean_arr, res_std_arr

In [4]:
# listp = [0.25,1,4,9,39,99]
# listL = [30,50,70,100]
listp = [99]
listL = [100]
t_t = 10000
n_iter = 1000
learning_rate = 0.00001
ddpg_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        res_mean, res_std = ddpg_eval_interval(p,L,t_t, n_iter, learning_rate)
        ddpg_res = ddpg_res.append({'p': p, 'L':L, 'res_mean':min(res_mean), 'res_std': res_std[np.argmin(res_mean)]}, ignore_index=True)

Running DDPG w/: p= 99 , L= 100
Using cpu device


KeyboardInterrupt: 

In [ ]:
ppo_res

In [ ]:
plt.pyplot.plot(res_mean)

## Below is obsolete.

In [7]:
ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
cont_env = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
print("Running DDPG w/: p=", p, ", L=",L)
n_actions = cont_env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

cont_model = DDPG("MlpPolicy", cont_env, action_noise=action_noise, verbose=1, gamma = 1)
cont_model.learn(total_timesteps=2)


Running DDPG w/: p= 99 , L= 100
Using cpu device


KeyboardInterrupt: 